In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [2]:
# Options for example codes
frame_enable = True

# Ex1) P2P example

In [3]:
robot = rob.Robot("indy7")
link_name = 6
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 0, 1, 1], [0, 1, 0, -0.1], [-1, 0, 0, 0.6], [0, 0, 0, 1]]
    ))
q0 = [1.0, -0.05311582280659044, -0.815452580946695, -1.3191046402052224, -0.8582660722530533, 1.3988994390898029]

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [4]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [5]:
#task_P2P.write_task_graph("Indy7_P2P_Task.svg")

In [6]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [7]:
#task_P2P.write_task_graph("Indy7_P2P_Task2.svg")

In [8]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      452
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      141

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      141
Total number of inequality constrai

In [9]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)
frameID0 = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)
robotIDs=[robotID,frameID0]

if frame_enable:
    frameIDs = [0]*horizon_steps
    for i in range(horizon_steps):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)


for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 3, 4, 5]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])

if frame_enable:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0])
    obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

In [10]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )
    
sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [3]:
robot = rob.Robot("indy7")
link_name = 6

# Define initial conditions of the robot
q0_val = [0, -0.423598, -0.9, 0.0, -1.5709, -0.523598]
qd0_val = [0] * robot.ndof

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
#task_P2P.write_task_graph("Indy7_P2P_Task4.svg")

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [4]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_indy7'])
task_P2P.remove_terminal_constraints('rot_con_pose_6_indy7_vs_goal',
                                    'trans_con_pose_6_indy7_vs_goal')
#task_P2P.write_task_graph("Indy7_P2P_Task5.svg")

In [5]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_indy7']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_6_indy7_vs_goal'], 5e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_6_indy7_vs_goal'], 1e0), 
                            Regularization(task_P2P.variables['qd_indy7'], 1e-2),
                            Regularization(task_P2P.variables['qdd_indy7'], 8e-5))

#task_P2P.write_task_graph("Indy7_P2P_Task6.svg")

In [6]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
mpc_options = default_mpc_options.get_default_mpc_options()

tc.ocp_solver = mpc_options["ipopt_init"]["solver_name"]
tc.ocp_options = mpc_options["ipopt_init"]["options"]
tc.mpc_solver = mpc_options["ipopt_lbfgs"]["solver_name"]
tc.mpc_options =mpc_options["ipopt_lbfgs"]["options"]
tc.set_ocp_solver(tc.ocp_solver, tc.ocp_options)

################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_indy7'].x
qd = pOCP.stage_tasks[0].variables['qd_indy7'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_indy7'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_indy7'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_indy7'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([-1, 0, 0, 0.5]),
    cs.hcat([0, 1, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_indy7", "var":"qd_indy7", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_6_indy7_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

# os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 188.00us ( 23.50us) 186.29us ( 23.29us)         8
       nlp_g  |  60.00us (  7.50us)  60.96us (  7.62us)         8
    nlp_grad  |  80.00us ( 80.00us)  80.06us ( 80.06us)         1
  nlp_grad_f  | 594.00us ( 66.00us) 529.79us ( 58.87us)         9
  nlp_hess_l  |   2.22ms (316.71us)   2.21ms (316.26us)         7
   nlp_jac_g  |  89.00us (  9.89us)  87.46us (  9.72us)         9
       total  |   5.98ms (  5.98ms)   5.88ms (  5.88ms)         1


In [7]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti0_p_2[12:15]-fk_T(opti0_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [8]:
MPC_component = MPC("indy7_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [9]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_green.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")

# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)
q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size


In [22]:
# reset cube velocity
p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# p.resetBaseVelocity(cubeID, linearVelocity=[-1, 0, 0])
# p.resetBaseVelocity(cubeID, linearVelocity=[1, 0, 0])

In [23]:
# Execute the MPC loop

for i in range(horizon_size * 10):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_indy7"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_indy7"]["val"] = qd_now

    # the position of the target object (cube)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)

    predicted_pos = lin_pos
    predicted_pos_log.append(predicted_pos.full())
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([-1, 0, 0, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_indy7"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_indy7"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_indy7"]["val"] * t_mpc).full()
    
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig+qdd_control_sig
    )
    
    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.49999107196306, 1.0450001091138, 0.084988581374053]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 179.00us ( 29.83us) 181.62us ( 30.27us)         6
       nlp_g  |  62.00us ( 10.33us)  61.74us ( 10.29us)         6
    nlp_grad  | 117.00us (117.00us) 113.17us (113.17us)         1
  nlp_grad_f  | 542.00us ( 77.43us) 543.51us ( 77.64us)         7
  nlp_hess_l  |   8.18ms (  1.64ms)   2.03ms (406.72us)         5
   nlp_jac_g  | 100.00us ( 14.29us)  99.43us ( 14.20us)         7
       total  |  20.04ms ( 20.04ms)   5.88ms (  5.88ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 118.00us ( 23.60us) 117.91us ( 23.58us)         5
       nlp_g  |  37.00us (  7.40us)  38.25us (  7.65us)         5
    nlp_grad  | 957.00us (957.00us)  85.28us ( 85.28us)         1
  nlp_grad_f  | 362.00us ( 60.33us) 365.16us ( 60.86us)         6
  nlp_hess_l  |   9.30

----------- MPC execution -----------
Predicted position of cube [0.49999106959864, 0.66244630587605, 0.084989999811693]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.93ms (321.67us) 181.37us ( 30.23us)         6
       nlp_g  |  42.00us (  7.00us)  42.09us (  7.02us)         6
    nlp_grad  | 122.00us (122.00us) 122.67us (122.67us)         1
  nlp_grad_f  | 528.00us ( 75.43us) 528.46us ( 75.49us)         7
  nlp_hess_l  |  10.02ms (  2.00ms)   2.02ms (404.21us)         5
   nlp_jac_g  | 100.00us ( 14.29us)  99.12us ( 14.16us)         7
       total  |  15.56ms ( 15.56ms)   5.78ms (  5.78ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999107598763, 0.63913241944901, 0.084989999930766]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 178.00us ( 29.67us) 178.58us ( 29.76us)         6
       nlp_g  |  48.00us (  8.00us)  48.37us (  8.06us)         6
    nlp_grad  |   1.95ms (  1.95

----------- MPC execution -----------
Predicted position of cube [0.49999112592395, 0.44205126460874, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 179.00us ( 29.83us) 178.10us ( 29.68us)         6
       nlp_g  |  52.00us (  8.67us)  53.49us (  8.92us)         6
    nlp_grad  | 109.00us (109.00us) 108.89us (108.89us)         1
  nlp_grad_f  | 523.00us ( 74.71us) 522.46us ( 74.64us)         7
  nlp_hess_l  |  10.02ms (  2.00ms)   2.03ms (405.66us)         5
   nlp_jac_g  |  98.00us ( 14.00us)  97.35us ( 13.91us)         7
       total  |  13.64ms ( 13.64ms)   5.65ms (  5.65ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999112657301, 0.43891289607867, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 180.00us ( 30.00us) 182.30us ( 30.38us)         6
       nlp_g  |  54.00us (  9.00us)  52.78us (  8.80us)         6
    nlp_grad  | 103.00us (103.00us) 103.54us (103.54

----------- MPC execution -----------
Predicted position of cube [0.49999115250973, 0.43633260092263, 0.084989999999912]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  17.50ms (  2.92ms) 236.39us ( 39.40us)         6
       nlp_g  |  61.00us ( 10.17us)  60.08us ( 10.01us)         6
    nlp_grad  | 120.00us (120.00us) 121.15us (121.15us)         1
  nlp_grad_f  |   1.18ms (169.00us) 679.00us ( 97.00us)         7
  nlp_hess_l  |  13.60ms (  2.72ms)   2.28ms (456.39us)         5
   nlp_jac_g  | 136.00us ( 19.43us) 122.05us ( 17.44us)         7
       total  |  67.85ms ( 67.85ms)   7.13ms (  7.13ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999115250979, 0.43633260092265, 0.08498999999988]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.08ms (179.67us) 199.30us ( 33.22us)         6
       nlp_g  | 178.00us ( 29.67us)  56.74us (  9.46us)         6
    nlp_grad  | 102.00us (102.00u

----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 460.00us ( 76.67us) 170.91us ( 28.49us)         6
       nlp_g  |  94.00us ( 15.67us)  56.25us (  9.38us)         6
    nlp_grad  | 782.00us (782.00us) 112.08us (112.08us)         1
  nlp_grad_f  |   1.30ms (186.00us) 532.40us ( 76.06us)         7
  nlp_hess_l  |   9.71ms (  1.94ms)   1.73ms (345.36us)         5
   nlp_jac_g  |  91.00us ( 13.00us)  89.48us ( 12.78us)         7
       total  |  18.13ms ( 18.13ms)   5.16ms (  5.16ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 234.00us ( 39.00us) 236.21us ( 39.37us)         6
       nlp_g  |  67.00us ( 11.17us)  67.26us ( 11.21us)         6
    nlp_grad  |   2.07ms (  2.07

----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 341.00us ( 56.83us) 278.37us ( 46.40us)         6
       nlp_g  |  83.00us ( 13.83us)  80.83us ( 13.47us)         6
    nlp_grad  | 213.00us (213.00us)  89.93us ( 89.93us)         1
  nlp_grad_f  |   4.84ms (691.14us) 815.31us (116.47us)         7
  nlp_hess_l  |  37.01ms (  7.40ms)   3.49ms (698.60us)         5
   nlp_jac_g  | 150.00us ( 21.43us) 149.38us ( 21.34us)         7
       total  |  92.52ms ( 92.52ms)   9.52ms (  9.52ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 282.00us ( 47.00us) 282.39us ( 47.07us)         6
       nlp_g  |  95.00us ( 15.83us)  95.24us ( 15.87us)         6
    nlp_grad  | 503.00us (503.00

----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 191.00us ( 31.83us) 190.66us ( 31.78us)         6
       nlp_g  |  55.00us (  9.17us)  54.58us (  9.10us)         6
    nlp_grad  | 132.00us (132.00us) 132.51us (132.51us)         1
  nlp_grad_f  | 569.00us ( 81.29us) 569.18us ( 81.31us)         7
  nlp_hess_l  |   2.02ms (404.60us)   2.03ms (405.35us)         5
   nlp_jac_g  |  99.00us ( 14.14us)  99.53us ( 14.22us)         7
       total  |  22.57ms ( 22.57ms)   5.84ms (  5.84ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999115251013, 0.43633260092275, 0.084989999999667]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 190.00us ( 31.67us) 190.38us ( 31.73us)         6
       nlp_g  |  62.00us ( 10.33us)  60.58us ( 10.10us)         6
    nlp_grad  | 124.00us (124.00

In [20]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.010145425796508789